In [1]:
# installing different versions of imbalanced learn and scikit in order to get the balanced random forest classifier to run
!pip install imbalanced-learn==0.9.0 
!pip install scikit-learn==1.0

  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.7.0
    Uninstalling imbalanced-learn-0.7.0:
      Successfully uninstalled imbalanced-learn-0.7.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.9.0 requires scikit-learn>=1.0.1, but you have scikit-learn 1.0 which is incompatible.



  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from collections import Counter

In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [5]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = 'resources/LoanStats_2019Q1.csv'
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
# custom encoding step
# loan_status, since it's our target variable and i want that as one column
# issue_d and next_pymnt_d, since i want to give them sequential numbers for the months (not strictly necessary but will look nicer)
issue_date = {
    'Jan-2019' : 0,
    'Feb-2019' : 1,
    'Mar-2019' : 2
}

next_payment_date = {
    'Apr-2019' : 0,
    'May-2019': 1
}

loan_status_encoding = {
    'low_risk' : 0,
    'high_risk' : 1
}

df["issue_date"] = df["issue_d"].apply(lambda x: issue_date[x])
df["next_payment"] = df["next_pymnt_d"].apply(lambda x: next_payment_date[x])
df["target"] = df["loan_status"].apply(lambda x: loan_status_encoding[x])

df = df.drop(["issue_d", "next_pymnt_d", "loan_status"], axis=1)

In [8]:
# get dummies encoding step
cols = list(df.select_dtypes(include='object'))
df = pd.get_dummies(df, columns=cols)

print(df.dtypes.value_counts())
df.head()

float64    76
uint8      14
int64       3
dtype: int64


,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,0,1,0,1,1,0,1,1


# Split the Data into Training and Testing

In [9]:
# Create our features
X = df.drop("target", axis=1)

# Create our target
y = df["target"]

In [10]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.478007,0.373992,0.148001,1.0,0.123879,0.876121,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499520,0.483865,0.355104,0.0,0.329446,0.329446,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [11]:
# Check the balance of our target values
y.value_counts()

0    68470
1      347
Name: target, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, stratify=y)

In [13]:
# resampling X data
from sklearn.preprocessing import StandardScaler

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [16]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

In [17]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7609523751969092

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[15427,  1691],
       [   33,    54]], dtype=int64)

In [19]:
# Print the imbalanced classification report
target_names = ["low_risk", "high_risk"]
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred, target_names= target_names))

                   pre       rec       spe        f1       geo       iba       sup

   low_risk       1.00      0.90      0.62      0.95      0.75      0.58     17118
  high_risk       0.03      0.62      0.90      0.06      0.75      0.54        87

avg / total       0.99      0.90      0.62      0.94      0.75      0.57     17205



In [21]:
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_, X.columns), reverse = True)

[(0.06984378249202676, 'total_rec_prncp'),
 (0.062422485871586705, 'total_rec_int'),
 (0.06210331692753507, 'total_pymnt_inv'),
 (0.06114239980617025, 'total_pymnt'),
 (0.059121967006725444, 'last_pymnt_amnt'),
 (0.030053139647431844, 'int_rate'),
 (0.024504761907907575, 'issue_date'),
 (0.017902944213186715, 'mo_sin_old_rev_tl_op'),
 (0.01771789452922284, 'annual_inc'),
 (0.016982824494712605, 'bc_util'),
 (0.016950354930327958, 'out_prncp_inv'),
 (0.016734732614059254, 'bc_open_to_buy'),
 (0.016307657958234318, 'max_bal_bc'),
 (0.015892171314835116, 'out_prncp'),
 (0.015772417716193716, 'revol_bal'),
 (0.015043165298060858, 'tot_hi_cred_lim'),
 (0.014764464451437276, 'tot_cur_bal'),
 (0.014516045137326936, 'avg_cur_bal'),
 (0.014502393983205656, 'installment'),
 (0.014472396106196042, 'mths_since_recent_inq'),
 (0.014434286731062757, 'total_il_high_credit_limit'),
 (0.014179783412218119, 'dti'),
 (0.014132458087708888, 'mo_sin_old_il_acct'),
 (0.013782601534743758, 'mths_since_rcnt_i

### Easy Ensemble AdaBoost Classifier

In [22]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

In [23]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9305540447441893

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[15921,  1197],
       [    6,    81]], dtype=int64)

In [25]:
# Print the imbalanced classification report
target_names = ["low_risk", "high_risk"]
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred, target_names= target_names))

                   pre       rec       spe        f1       geo       iba       sup

   low_risk       1.00      0.93      0.93      0.96      0.93      0.87     17118
  high_risk       0.06      0.93      0.93      0.12      0.93      0.87        87

avg / total       0.99      0.93      0.93      0.96      0.93      0.87     17205

